# Libraries importing

In [38]:
import numpy as np
import os
import pickle
import tensorflow as tf
from tensorflow_model_optimization.quantization.keras import vitis_inspect, vitis_quantize
from tensorflow_model_optimization.python.core.quantization.keras.vitis.utils import model_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, SGD

# Function to load the CIFAR10 dataset (after downloading it locally)

In [2]:
def load_cifar10(data_dir):
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict

    # Load training data
    x_train, y_train = [], []
    for i in range(1, 6):  # CIFAR-10 training data consists of 5 batches
        batch = unpickle(os.path.join(data_dir, f"data_batch_{i}"))
        x_train.append(batch[b'data'])
        y_train.extend(batch[b'labels'])

    x_train = np.vstack(x_train).reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    y_train = np.array(y_train)

    # Load test data
    test_batch = unpickle(os.path.join(data_dir, "test_batch"))
    x_test = test_batch[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    y_test = np.array(test_batch[b'labels'])

    return (x_train, y_train), (x_test, y_test)

# Actually loading CIFAR10

In [11]:
#CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = load_cifar10("cifar-10-batches-py")
print(train_images.shape)
train_images = train_images.reshape(train_images.shape[0], 32, 32, 3) #ensure shape 32 W x 32 H x 3 channels for each image
test_images = test_images.reshape(test_images.shape[0], 32, 32, 3)

#range 0-1
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

#One-hot encoding labels
train_labels = to_categorical(test_images, 10)
test_labels = to_categorical(test_labels, 10)

(50000, 32, 32, 3)


# Loading a trained model

In [26]:
input_shape = (None, 32, 32, 3)
model = load_model('best_model.h5')
model.build(input_shape=input_shape)
model.summary()

opt = SGD(learning_rate=0.1, momentum=0.9, weight_decay=1e-4)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 64)   1792        ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 re_lu (ReLU)                   (None, 32, 32, 64)   0           ['batch_normalization[0][0]']

# Copying the guide, 17:57 in the video

In [39]:
target_dpu = 'DPUCZDX8G_ISA1_B4096'
inspector = vitis_inspect.VitisInspector(target = target_dpu)

if not os.path.exists("inspect"):
    os.makedirs("inspect")

inspector.inspect_model(model,
                        input_shape = input_shape,
                        dump_model = True,
                        dump_model_file = "inspect/inspected_model.h5",
                        plot = True,
                        plot_file = "inspect/model_plot.svg",
                        dump_results = True,
                        dump_results_file = "inspect/inspect_results.txt",
                        verbose = 0)

[VAI INFO] Update include_bias_corr: False
[VAI INFO] Update include_fast_ft: False
[VAI INFO] Update include_cle: False
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/zeart/tensorflow_model_optimization/tmp_sub_graph_quantized_20241227-095425-034272_WkwFUZ.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/zeart/tensorflow_model_optimization/tmp_sub_graph_xnnc_20241227-095425-034272_WkwFUZ.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/zeart/tensorflow_model_optimization/tmp_sub_graph_quantized_20241227-095425-034272_WkwFUZ.h5
[INFO] keras version: 2.12.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|███████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 3177.50it/s]                  

[INFO] dump xmodel: /tmp/zeart/tensorflow_model_optimization/tmp_sub_graph_xnnc_20241227-095425-034272_WkwFUZ.xmodel



I20241227 09:54:28.328120   109 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20241227 09:54:28.328169   109 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20241227 09:54:28.328176   109 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B4096
I20241227 09:54:28.328691   109 compile_pass_manager.cpp:465] [UNILOG][INFO] Graph name: model_248, with op num: 6
I20241227 09:54:28.328699   109 compile_pass_manager.cpp:478] [UNILOG][INFO] Begin to compile...
I20241227 09:54:28.333644   109 compile_pass_manager.cpp:489] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20241227 09:54:28.333714   109 compile_pass_manager.cpp:504] [UNILOG][INFO] Compile done.


[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/zeart/tensorflow_model_optimization/tmp_sub_graph_quantized_20241227-095425-035079_atwPEy.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/zeart/tensorflow_model_optimization/tmp_sub_graph_xnnc_20241227-095425-035079_atwPEy.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/zeart/tensorflow_model_optimization/tmp_sub_graph_quantized_20241227-095425-035079_atwPEy.h5
[INFO] keras version: 2.12.0
[INFO] Tensorflow Keras model type: functional


...

[INFO] generate xmodel     :100%|███████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 4230.57it/s]                  
I20241227 09:54:37.415491   109 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20241227 09:54:37.415549   109 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20241227 09:54:37.415558   109 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B4096
I20241227 09:54:37.415653   109 compile_pass_manager.cpp:465] [UNILOG][INFO] Graph name: model_277, with op num: 7
I20241227 09:54:37.415661   109 compile_pass_manager.cpp:478] [UNILOG][INFO] Begin to compile...
I20241227 09:54:37.423898   109 compile_pass_manager.cpp:489] [UNILOG][INFO] Total device subgraph number 4, DPU subgraph number 1
I20241227 09:54:37.423969   109 compile_pass_manager.cpp:504] [UNILOG][INFO] Compile done.


[INFO] dump xmodel: /tmp/zeart/tensorflow_model_optimization/tmp_sub_graph_xnnc_20241227-095425-035444_WACYiB.xmodel
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/zeart/tensorflow_model_optimization/tmp_sub_graph_quantized_20241227-095425-035444_WACYiB.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/zeart/tensorflow_model_optimization/tmp_sub_graph_xnnc_20241227-095425-035444_WACYiB.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/zeart/tensorflow_model_optimization/tmp_sub_graph_quantized_20241227-095425-035444_WACYiB.h5
[INFO] keras version: 2.12.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|███████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 3168.59it/s]                  
I20241227 09:54:37.593210   109 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20241227 09:54:37.593268   109 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20241227 09:54:37.593282   109 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B4096
I20241227 09:54:37.593432   109 compile_pass_manager.cpp:465] [UNILOG][INFO] Graph name: model_278, with op num: 7
I20241227 09:54:37.593449   109 compile_pass_manager.cpp:478] [UNILOG][INFO] Begin to compile...
I20241227 09:54:37.596632   109 compile_pass_manager.cpp:489] [UNILOG][INFO] Total device subgraph number 4, DPU subgraph number 1
I20241227 09:54:37.596670   109 compile_pass_manager.cpp:504] [UNILOG][INFO] Compile done.


[INFO] dump xmodel: /tmp/zeart/tensorflow_model_optimization/tmp_sub_graph_xnnc_20241227-095425-035444_WACYiB.xmodel
[VAI INFO] Inspect Results:
[MODEL INFO]:
________________________________________________________________________________________________________________________
Model Name: model
________________________________________________________________________________________________________________________
ID          Name                    Type                    Device      Notes                                           
0/67        input_313               InputLayer              INPUT                                                       
------------------------------------------------------------------------------------------------------------------------
1/67        conv2d                  Conv2D<relu>            DPU                                                         
-------------------------------------------------------------------------------------------------

{<keras.engine.input_layer.InputLayer at 0x7f006c6c8520>: <tensorflow_model_optimization.python.core.quantization.keras.vitis.vitis_inspect.InspectResult at 0x7f004b9e9e20>,
 <keras.layers.convolutional.conv2d.Conv2D at 0x7f006c6c8760>: <tensorflow_model_optimization.python.core.quantization.keras.vitis.vitis_inspect.InspectResult at 0x7f004b9b2250>,
 <keras.layers.normalization.batch_normalization.BatchNormalization at 0x7f006c6c8580>: <tensorflow_model_optimization.python.core.quantization.keras.vitis.vitis_inspect.InspectResult at 0x7f004b9b2eb0>,
 <keras.layers.activation.relu.ReLU at 0x7f006c31c790>: <tensorflow_model_optimization.python.core.quantization.keras.vitis.vitis_inspect.InspectResult at 0x7f004b9b2490>,
 <keras.layers.convolutional.conv2d.Conv2D at 0x7f004bb82ac0>: <tensorflow_model_optimization.python.core.quantization.keras.vitis.vitis_inspect.InspectResult at 0x7f004b9b2880>,
 <keras.layers.normalization.batch_normalization.BatchNormalization at 0x7f004bb829a0>: <ten

# Quantize model

In [ ]:
calib_data = train_images[:100]
# Create a tf.data.Dataset and ensure batch dimension
dataset = tf.data.Dataset.from_tensor_slices(calib_data) # convert 100,32,32,3 to a 100 item iterable/list
""" for item in dataset:
    print(item) """
# no need for data calibration because train_images was already calibrated earlier
dataset = dataset.batch(5) # group 5 images each time to check activation and weight distribution

# Inspect calibration data
""" for sample in dataset.take(100):
    print("Calibration sample shape:", sample.shape)
    print("Sample data range:", sample.numpy().min(), sample.numpy().max()) """

# Quantization
try:
    quantizer = vitis_quantize.VitisQuantizer(model, target=target_dpu) #int8 accordiing to docs
    q_model = quantizer.quantize_model(calib_dataset=dataset, 
                                       input_shape=input_shape, 
                                       calib_steps=None) #use all of the dataset
    q_model.save("quantized_model.h5")
    print("Quantized model saved successfully.")
except Exception as e:
    print("Error during quantization:", e)

[VAI INFO] Using func format quantizer
[VAI INFO] Quantizing without specific `target`.
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 11s 1s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...


2024-12-27 10:38:33.330936: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [100,32,32,3]
	 [[{{node Placeholder/_0}}]]


20/20 [==============================] - 41s 847ms/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quant Model Refinement...
[VAI INFO] Start Quantize Position Ajustment...
[VAI INFO] Quantize Position Ajustment Done.
1/1 [==============================] - 2s 2s/step
[VAI INFO] Post-Quant Model Refninement Done.
[VAI INFO] Start Model Finalization...
[VAI INFO] Model Finalization Done.
[VAI INFO] Quantization Finished.
Quantized model saved successfully.


# Recompile & evaluate quantized model

In [28]:
batch_size = 128

#i dont think we need this - Zé 

#learning_rate = 0.0001
#momentum = 0
#epsilon = 1e-08

""" q_model.compile(
    optimizer = tf.keras.optimizers.RMSprop(
        learning_rate = learning_rate,
        momentum = momentum,
        epsilon = epsilon),
    loss = tf.keras.losses.CategoricalCrossentropy(reduction = tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE),
    metrics = ['acc'])
 """

# Instead:
#compile with a dummy optimizer, since we are not training 
q_model.compile(optimizer='sgd',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

q_model.evaluate(test_images, test_labels, batch_size=batch_size)

# quantizer.dump_model(q_model, dataset = test_images, dump_float = True)

2024-12-27 06:51:46.857828: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.


 8/79 [==>...........................] - ETA: 1:45 - loss: 7.1162 - acc: 0.1123

KeyboardInterrupt: 

# Compile the DPU

In [ ]:
vai_c_tensorflow2 -m "/home/s3515850/EAI/.venv/quantized_model.h5" -a "/opt/vitis_ai/compiler/arch/DPUCZDX8G/KV260/arch.json" -o "/home/s3515850/EAI/.venv/xmodel" -n kria_model

/bin/bash: vai_c_tensorflow2: command not found
